In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
%matplotlib inline
import matplotlib.pyplot as plt


In [4]:
# Load the data
# file_path = Path('Average Temperature of Cities.csv')
file_path = Path('Annual Average AZ Temps 1970-2022.csv')
file = Path('AZ CO2 Emissions Per Capita.csv')
df = pd.read_csv(file_path)

In [5]:
df_file = pd.read_csv(file)
df_file.head()

,State,Year,Metric Tons of Energy-Related CO2 per person
0,Arizona,1970,13.9
1,Arizona,1971,14.2
2,Arizona,1972,15.0
3,Arizona,1973,16.2
4,Arizona,1974,16.5


In [6]:
df = df.drop(columns = 'State')
df.head()

,Year,Degrees Fahrenheit,Anomaly
0,1970,59.0,-0.8
1,1971,58.2,-1.6
2,1972,59.6,-0.2
3,1973,58.4,-1.4
4,1974,59.5,-0.3


In [7]:
df_file = df_file.drop(columns = 'State')

In [8]:
df_file.head()

,Year,Metric Tons of Energy-Related CO2 per person
0,1970,13.9
1,1971,14.2
2,1972,15.0
3,1973,16.2
4,1974,16.5


In [11]:
df = pd.merge(df, df_file, on = 'Year')

In [12]:
df.head()

,Year,Degrees Fahrenheit,Anomaly,Metric Tons of Energy-Related CO2 per person
0,1970,59.0,-0.8,13.9
1,1971,58.2,-1.6,14.2
2,1972,59.6,-0.2,15.0
3,1973,58.4,-1.4,16.2
4,1974,59.5,-0.3,16.5


In [13]:
df.columns

Index(['Year', 'Degrees Fahrenheit', 'Anomaly',
       'Metric Tons of Energy-Related CO2 per person'],
      dtype='object')

In [14]:
X = df[['Year', 'Anomaly',
       'Metric Tons of Energy-Related CO2 per person']].copy()

X = X.dropna()
y=df['Degrees Fahrenheit']
X.head()

,Year,Anomaly,Metric Tons of Energy-Related CO2 per person
0,1970,-0.8,13.9
1,1971,-1.6,14.2
2,1972,-0.2,15.0
3,1973,-1.4,16.2
4,1974,-0.3,16.5


In [15]:
X.dropna()

,Year,Anomaly,Metric Tons of Energy-Related CO2 per person
0,1970,-0.8,13.90
1,1971,-1.6,14.20
2,1972,-0.2,15.00
3,1973,-1.4,16.20
4,1974,-0.3,16.50
5,1975,-1.9,16.70
6,1976,-0.8,18.70
7,1977,0.4,20.80
8,1978,0.0,19.60
9,1979,-1.0,21.30


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [17]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X_train, y_train)

score = model.score(X, y)
print(f"R2 Score: {score}")

R2 Score: 1.0


In [18]:
y_pred = model.predict(X_test)

In [19]:
y_pred

array([59.2, 61.1, 61.7, 61.8, 59.3, 62.3, 61.9, 61. , 61.2, 61.2, 58.7,
       59.5, 61.6])

In [20]:
model.predict([2050, 1.8, 13.10])

ValueError: Expected 2D array, got 1D array instead:
array=[2.05e+03 1.80e+00 1.31e+01].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [22]:
df.columns

Index(['Year', 'Degrees Fahrenheit', 'Anomaly',
       'Metric Tons of Energy-Related CO2 per person'],
      dtype='object')

In [24]:
test = pd.DataFrame({'Year':2050, 'Anomaly': 1.5,
       'Metric': 13.10})

ValueError: If using all scalar values, you must pass an index